In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving nxt.txt to nxt.txt


In [ ]:
file = open("nxt.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of The Prince This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title: The Prin'

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[1, 78, 162, 488, 4, 1, 34, 22, 488, 17, 15, 1, 187, 4, 1032]

In [ ]:
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]


5593
The Length of sequences are:  53214


array([[  1,  78, 162, 488],
       [ 78, 162, 488,   4],
       [162, 488,   4,   1],
       [488,   4,   1,  34],
       [  4,   1,  34,  22],
       [  1,  34,  22, 488],
       [ 34,  22, 488,  17],
       [ 22, 488,  17,  15],
       [488,  17,  15,   1],
       [ 17,  15,   1, 187]])

In [ ]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

Data:  [[  1  78 162]
 [ 78 162 488]
 [162 488   4]
 [488   4   1]
 [  4   1  34]
 [  1  34  22]
 [ 34  22 488]
 [ 22 488  17]
 [488  17  15]
 [ 17  15   1]]
Response:  [488   4   1  34  22 488  17  15   1 187]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
832/832 [==============================] - ETA: 0s - loss: 6.4873
Epoch 1: loss improved from inf to 6.48726, saving model to next_words.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


832/832 [==============================] - 27s 23ms/step - loss: 6.4873
Epoch 2/70
832/832 [==============================] - ETA: 0s - loss: 5.9671
Epoch 2: loss improved from 6.48726 to 5.96708, saving model to next_words.h5
832/832 [==============================] - 14s 17ms/step - loss: 5.9671
Epoch 3/70
830/832 [============================>.] - ETA: 0s - loss: 5.5688
Epoch 3: loss improved from 5.96708 to 5.56829, saving model to next_words.h5
832/832 [==============================] - 13s 16ms/step - loss: 5.5683
Epoch 4/70
832/832 [==============================] - ETA: 0s - loss: 5.2686
Epoch 4: loss improved from 5.56829 to 5.26856, saving model to next_words.h5
832/832 [==============================] - 13s 16ms/step - loss: 5.2686
Epoch 5/70
832/832 [==============================] - ETA: 0s - loss: 5.0280
Epoch 5: loss improved from 5.26856 to 5.02796, saving model to next_words.h5
832/832 [==============================] - 13s 16ms/step - loss: 5.0280
Epoch 6/70
832/832 [

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue



Enter your line: His life falls
['His', 'life', 'falls']
1/1 [==============================] - 1s 627ms/step
naturally
Enter your line: activity and increasing
['activity', 'and', 'increasing']
1/1 [==============================] - 0s 18ms/step
influence
